In [1]:
import pandas as pd
import xarray as xr

from scipy import optimize as opt
from scipy import stats

In [2]:
path = "../results/prec_miro_hist_hist_tretorno.nc"
path_1 = "../results/prec_miro_hist_hist_tretorno_1.nc"
df = xr.open_dataset(path).to_dataframe()

df.loc[(90, 90, 1), "PROBABILIDAD"]

TIEMPO_RETORNO
1.032258     0.96875
1.066667     0.93750
1.103448     0.90625
1.142857     0.87500
1.185185     0.84375
1.230769     0.81250
1.280000     0.78125
1.333333     0.75000
1.391304     0.71875
1.454545     0.68750
1.523810     0.65625
1.600000     0.62500
1.684211     0.59375
1.777778     0.56250
1.882353     0.53125
2.000000     0.50000
2.133333     0.46875
2.285714     0.43750
2.461538     0.40625
2.666667     0.37500
2.909091     0.34375
3.200000     0.31250
3.555556     0.28125
4.000000     0.25000
4.571429     0.21875
5.333333     0.18750
6.400000     0.15625
8.000000     0.12500
10.666667    0.09375
16.000000    0.06250
32.000000    0.03125
Name: PROBABILIDAD, dtype: float64

In [3]:
params = stats.genextreme.fit(df.loc[(90, 90, 1), "INTENSIDAD"])
stats.genextreme.sf(df.loc[(90, 90, 1), "INTENSIDAD"], *params)

array([0.98898712, 0.92303627, 0.89666945, 0.88345911, 0.84864468,
       0.82158113, 0.82100579, 0.80976824, 0.735389  , 0.72017908,
       0.66792676, 0.66666586, 0.66104862, 0.62814016, 0.62705607,
       0.6003613 , 0.36377392, 0.35847926, 0.31535846, 0.30010497,
       0.29663971, 0.25899195, 0.25034423, 0.24765579, 0.21353394,
       0.20342284, 0.15152248, 0.12482662, 0.04727001, 0.0438078 ,
       0.02327957])

In [5]:
params

(-0.08459465862054488, 20.053986783998646, 6.2893980125965125)

In [4]:
stats.kstest(df.loc[(90, 90, 1), "INTENSIDAD"],
    stats.genextreme(*params).cdf ).pvalue

0.7179545797090455